RAG Fusion with Local LLM.


Install python packages required for langchain

In [ ]:
! pip install -r ../requirements.txt

Set up os environment variables.
This is to enable visibility of tracing langchain invocations on smith.langchain.com

In [ ]:
import os

# Uncomment below if you want to use [LangSmith](https://smith.langchain.com/) for debugging
# os.environ['LANGCHAIN_TRACING_V2'] = 'true'
# os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
# os.environ['LANGCHAIN_API_KEY'] = "<LANGCHAIN_API"
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

Set up indexing for the vector store db

In [ ]:
from ragatouille import RAGPretrainedModel
from langchain_community.document_loaders import DirectoryLoader, UnstructuredPDFLoader, UnstructuredMarkdownLoader
from langchain.retrievers import ContextualCompressionRetriever
from langchain_text_splitters import RecursiveCharacterTextSplitter

# https://python.langchain.com/v0.2/docs/integrations/retrievers/ragatouille/
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

documents = DirectoryLoader("../documents/high_level_docs", glob="**/*.md", show_progress=True,
                            loader_cls=UnstructuredMarkdownLoader).load()
apidocs = DirectoryLoader("../documents/api_docs", glob="**/*.md", show_progress=True,
                          loader_cls=UnstructuredMarkdownLoader).load()
features = DirectoryLoader("../documents/cucumber_feature_files", glob="**/*.md", show_progress=True,
                           loader_cls=UnstructuredMarkdownLoader).load()
pdf_docs = DirectoryLoader('../documents/high_level_docs', glob="**/*.pdf", show_progress=True,
                           loader_cls=UnstructuredPDFLoader).load()

all_documents = []
# Extract the text from the loaded documents
documents_text = [doc.page_content for doc in documents]
apidocs_text = [doc.page_content for doc in apidocs]
features_text = [doc.page_content for doc in features]
pdf_docs_text = [doc.page_content for doc in pdf_docs]
all_documents.extend(documents_text)
all_documents.extend(apidocs_text)
all_documents.extend(features_text)
all_documents.extend(pdf_docs_text)

RAG.index(
    collection=all_documents,
    index_name="Combined-Docs",
    max_document_length=512,
    split_documents=True,
)

retriever = RAG.as_langchain_retriever()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=RAG.as_langchain_document_compressor(), base_retriever=retriever
)


Reranker ColBERT with context about the question

In [ ]:
question="What fields are included in the request to Purchase Orchestrator?"

context_docs = compression_retriever.invoke(question)
print(f">>>>>>>>>>>>>>> DOC: \n\n" + context_docs[0].page_content)
print(f">>>>>>>>>>>>>>> SCORE: \n\n {context_docs[0].metadata}")
print(f">>>>>>>>>>>>>>> DOC: \n\n" + context_docs[1].page_content)
print(f">>>>>>>>>>>>>>> SCORE: \n\n {context_docs[1].metadata}")

Pass the documents back into the context for the LLM and include the original question
 and Try it out!

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": lambda x: "\n".join(doc.page_content for doc in context_docs[:2]), 
     "question": itemgetter("question")} 
    | prompt
    | ChatOllama(model="llama3")
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})